In [1]:
import numpy as np # Matris oluşturmaya yarayan kütüphane pythonda arrayleri plot etmemiz için yani müzik sinyal gösterimi ve daha birçok hesaplama için gerekli kütüphane
from math import (sin, pi)

import sys     # Sistem erişimi için kütüphane
import pygame          # Oyun yapmamızı sağlayan bir kütüphane ancak biz burda canvas çizimleri ve her işletim sisteminde müzik çalabilme özelliği için kullanıyoruz
from time import sleep,time      

from PyQt5.QtWidgets import (QDialog, QApplication, QWidget,
                             QVBoxLayout, QHBoxLayout,
                             QDesktopWidget, QFileDialog,                          # Grafik arayüzü(gui) oluşturmak için kullandığımız Pyqt5 kütüphaneleri
                             QSlider, QPushButton, QLabel, 
                             QCheckBox, QLCDNumber)
from PyQt5.QtCore import (Qt, QRunnable, pyqtSlot, QThreadPool)
from PyQt5.QtGui import QPixmap

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar        # Müzik sinyallerini plot etmemizi sağlayan kütüphaneler
import matplotlib.pyplot as plt 
import wave
            
        
class Worker(QRunnable):                                #Burada bir işçi sınıfı oluşturuyoruz daha sonra her işçi sınıfına bir slider atayacağız ve bu                                                                      sliderlardaki değeri kullanarak sinyali işleyecek
    def __init__(self, function, *args, **kwargs):
        super(Worker, self).__init__()
        self.function = function
        self.args = args
        self.kwargs = kwargs
        

    @pyqtSlot() # pytqt görselleştirme için kullandığımız arayüzde müziğin çalarken işlem bölünmeden tuşlara basabilmemiz için slot protokolü oluşturduk
    def run(self):
        self.function(*self.args, **self.kwargs)



class Main_Window_class(QDialog):   #Grafik arayüzü için açtığımız ana menü sınıfı tüm veriler burada alınır butonlar burada belirlenir. Kısacası arayüzü                                              ilgilendiren her işlem buraya bağlıdır
    def __init__(self):
        super().__init__()
        self.initUI()

        
    def initUI(self):
        ### Hyperparameters ###
        self.nlabels = 6
        
        self.checkboxes_lables = ['Crop', 'Envelope']   # checkbox arayüzü
        self.btns_lables = ['Play', 'Pause', 'Stop']    # butonlar
        self.app_name = 'Equalizer App' # uygulama ismi
        
        self.sld_min = -50  #Burası inebileceğimiz minumum slider eşik değeri.
        self.sld_max = 50   # Ses sinyalinin maximum slider eşik katı.
        self.sld_def = 0        
        self.sld_interval = 10  # Gerekli başlangıç tanımlamaları
        self.sld_step = 1
        #######################
        
        ### Evrensel veriler ###
        self.path_to_pull = None
        
        self.nchannels = None        # Kanal satısı
        self.sampwidth = None        # Her verideki byte sayısı
        self.framerate = None        # saniyedeki frame sayısı
        self.nframes = None          # toplam frame sayısı
        self.comptype = None         # ses dosyası sıkıştırma türü
        self.compname = None         # ses dosyası sıkıştırma ismi
        self.elem_per_herz = None
        self.koeff = 1000            
        self.types = {}
        self.buffer_size = None
        self.buffer_cnt = 0
        
        self.music_is_playing = False   # Bu veri müziğin çalıp durdurulmasını tutar pause play ve stop fonksiyonları için çalma durumu bilgisini barındırır.
        self.threadpool = QThreadPool()    # Thread yapısı kullanarak aynı anda birden fazla işlem yapabiliriz. Bu fonksiyon sayesinde grafik arayüzü anlık veri gösterirken bizde butonlara basabikme etkileşimini gerçekleştirebiliyoruz
        print(self.threadpool.maxThreadCount())

        self.music_worker = None
        self.sld1_worker = None
        self.sld2_worker = None
        self.sld3_worker = None             # Başlangıç durumu için her sliderın workerlarını None tanımladık slider kaydırma durumunda bu değerler değişicek
        self.sld4_worker = None
        self.sld5_worker = None
        self.sld6_worker = None
        self.checkbox1_worker = None
        self.checkbox2_worker = None
        
        self.min_freq = 0           # Projede de geçtiği üzere başlangıç sinyal değerini sıfır belirledik
        self.max_freq = None
        
        self.channels = []
        self.spectrum = None
        self.spectrum_original = None   # buradaki tüm parametreler giriş için verilmiş olup sınıf yapısı oluşturmak için burada tutulmaktadır 
        self.spectrum_kliping = None
        self.spectrum_envelop = None
        self.channels_original = []
        self.channels_kliping = []
        self.channels_envelop = []
        
        self.bands = [[], []]
        self.labels = []
 

        
        self.path_to_pull = QFileDialog.getOpenFileName(self, 'wav dosyaları .wav ')[0] # Program başında müzik dosyasını çeken grafik arayüzü kodu 
            # Kodda sadece wav bdosyalarını çekmesini belirledik çünkü sıkıştırılmış mp3 gibi müziklerde sinyali direk alamayız. Bozuk bir cızırtı sesi gelir
        
        self.pull_music()
        self.create_bands()
        self.create_lables()
        self.create_LCD_numbers()
        self.create_sliders()           # her şey belirlendiğinde buradaki kod parçacıklaro ile arayüzü tamamiyle oluşturulur. 
        self.create_checkboxes()
        self.create_buttons()
        self.create_graphics()
        self.create_interface()
    
    
    def pull_music(self):
        wav = wave.open(self.path_to_pull, mode = 'r')      # Müziği çektiğimiz fonksiyon müzik int8,int16,int32 şekillerinde kanallı yapıda olabilir fonksiyonun çoğu müziği hatası alabilmesi için bu şekide parametreler belirlenmiştir 
        self.types = {
            1: np.int8,
            2: np.int16,
            4: np.int32
        }

        (self.nchannels, self.sampwidth,
         self.framerate, self.nframes,                          # çekilen wav dosyasından tüm parametreleri alır, bu değerler ile sdaha sorna işlemeler yapılacaktır
         self.comptype, self.compname) = wav.getparams()
        
        self.max_freq = self.framerate // 2
        self.buffer_size = self.framerate

        content = wav.readframes(self.nframes)      # wav dosyasının frame sayısını content değişkenine atadık
        samples = np.fromstring(content, dtype = self.types[self.sampwidth]) # birim zamanda taşınan örnek boyutunu sample değişkenine atadık.

        for i in range(self.nchannels):
            self.channels.append(samples[i::self.nchannels]) 
        self.channels = np.array(self.channels)
        
        self.channels_original = self.channels.copy()
        
        self.checkbox1_worker = Worker(self.doing_kliping, self.channels)   # checkboxlardaki kırma işlemleri için threadları çalıştırır
        self.threadpool.start(self.checkbox1_worker)
        
        self.checkbox2_worker = Worker(self.doing_envelop, self.channels)
        self.threadpool.start(self.checkbox2_worker)
        
        self.spectrum = np.fft.rfft(self.channels_original)
        self.spectrum_original = self.spectrum.copy()
        
        pygame.mixer.pre_init(frequency = self.framerate,
                              size = -self.sampwidth * 8,
                              channels = self.nchannels)
        pygame.init()
        
        
    def create_bands(self):
        step = (self.max_freq - self.min_freq) // 2**self.nlabels       # slider için adım boyutu belirlenir
        
        self.bands[0].append(self.min_freq)         # minumum frekans değerini band[0] a atadık
        self.bands[1].append(self.min_freq + 400)   # ilk slider için ikinci değeri girdik
        
        for i in range(1, self.nlabels - 1):
            self.bands[0].append(self.bands[1][i - 1])
            self.bands[1].append(self.bands[0][i] + 2**i * step)    # bu fonksiyon sliderlara göre eşit paylaşımlı band oluşturmaya yarıyor ancak bizim projemizde tüm bandler aralıkları belirli olduğu için burada sadece 5000 den sonrasındaki band için burayı kullanıyoruz
        
        self.bands[0].append(self.bands[1][self.nlabels - 2])
        self.bands[1].append(self.max_freq)

        self.bands[0][0]=0
        self.bands[1][0]=400
        self.bands[0][1]=400
        self.bands[1][1]=700
        self.bands[0][2]=700
        self.bands[1][2]=1500               # Tüm band değerlerini projemizde açıklanan şekilde girdik
        self.bands[0][3]=1500
        self.bands[1][3]=3000
        self.bands[0][4]=3000
        self.bands[1][4]=5000
        self.bands[0][5]=5000


        for i in range(self.nlabels):
            self.labels.append(str(self.bands[0][i]) + ' - ' + str(self.bands[1][i]))   # band değerlerini grafik arayüzüne dinamik şekilde yazdırıyoruz
    
    
    def create_lables(self):
        self.label_1 = QLabel(self.labels[0], self)
        self.label_2 = QLabel(self.labels[1], self)
        self.label_3 = QLabel(self.labels[2], self)
        self.label_4 = QLabel(self.labels[3], self)     # Grafik arayüzünde göstermek için qlabelları oluşturup içine band değerlerini atadık
        self.label_5 = QLabel(self.labels[4], self)
        self.label_6 = QLabel(self.labels[5], self)
    
    
    def create_LCD_numbers(self):
        self.num_1 = QLCDNumber(self)
        self.num_2 = QLCDNumber(self)
        self.num_3 = QLCDNumber(self)                   # yaptığımız slider seçimini aynı zamanda lcd ekranda gösterdik
        self.num_4 = QLCDNumber(self)
        self.num_5 = QLCDNumber(self)
        self.num_6 = QLCDNumber(self)
    
    
    def create_sliders(self):
        self.sld_1 = QSlider(Qt.Vertical, self)
        self.sld_2 = QSlider(Qt.Vertical, self)
        self.sld_3 = QSlider(Qt.Vertical, self)             
        self.sld_4 = QSlider(Qt.Vertical, self)
        self.sld_5 = QSlider(Qt.Vertical, self) #sliderları oluşturdk
        self.sld_6 = QSlider(Qt.Vertical, self)
        
        self.sld_1.setMinimum(self.sld_min)
        self.sld_2.setMinimum(self.sld_min)
        self.sld_3.setMinimum(self.sld_min) # min eşikleri kurduk
        self.sld_4.setMinimum(self.sld_min)
        self.sld_5.setMinimum(self.sld_min)
        self.sld_6.setMinimum(self.sld_min)
        
        self.sld_1.setMaximum(self.sld_max)
        self.sld_2.setMaximum(self.sld_max)
        self.sld_3.setMaximum(self.sld_max)
        self.sld_4.setMaximum(self.sld_max) # max eşikleri kurduk
        self.sld_5.setMaximum(self.sld_max)
        self.sld_6.setMaximum(self.sld_max)
        
        self.sld_1.setValue(self.sld_def)
        self.sld_2.setValue(self.sld_def)
        self.sld_3.setValue(self.sld_def)
        self.sld_4.setValue(self.sld_def)
        self.sld_5.setValue(self.sld_def)
        self.sld_6.setValue(self.sld_def)         # Slider tasasrımından belirlediğimiz band aralıklarını slider'a aktarmak gibi birçok ayarlamanın yapıldığı kısım
        
        self.sld_1.setFocusPolicy(Qt.StrongFocus)
        self.sld_2.setFocusPolicy(Qt.StrongFocus)
        self.sld_3.setFocusPolicy(Qt.StrongFocus)
        self.sld_4.setFocusPolicy(Qt.StrongFocus)
        self.sld_5.setFocusPolicy(Qt.StrongFocus)
        self.sld_6.setFocusPolicy(Qt.StrongFocus)
        
        self.sld_1.setTickPosition(QSlider.TicksBothSides)
        self.sld_2.setTickPosition(QSlider.TicksBothSides)
        self.sld_3.setTickPosition(QSlider.TicksBothSides)
        self.sld_4.setTickPosition(QSlider.TicksBothSides)
        self.sld_5.setTickPosition(QSlider.TicksBothSides)
        self.sld_6.setTickPosition(QSlider.TicksBothSides)
        
        self.sld_1.setTickInterval(self.sld_interval)
        self.sld_2.setTickInterval(self.sld_interval)   # aralıkları kurduk
        self.sld_3.setTickInterval(self.sld_interval)
        self.sld_4.setTickInterval(self.sld_interval)
        self.sld_5.setTickInterval(self.sld_interval)
        self.sld_6.setTickInterval(self.sld_interval)
        
        self.sld_1.setSingleStep(self.sld_step)
        self.sld_2.setSingleStep(self.sld_step) #stepleri kurduk
        self.sld_3.setSingleStep(self.sld_step)
        self.sld_4.setSingleStep(self.sld_step)
        self.sld_5.setSingleStep(self.sld_step)
        self.sld_6.setSingleStep(self.sld_step)
        
        self.sld_1.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_2.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_3.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_4.valueChanged[int].connect(self.sliderChangeValue) # değişim değerini kurduk
        self.sld_5.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_6.valueChanged[int].connect(self.sliderChangeValue)
        
        self.sld_1.valueChanged[int].connect(self.num_1.display)
        self.sld_2.valueChanged[int].connect(self.num_2.display)
        self.sld_3.valueChanged[int].connect(self.num_3.display)
        self.sld_4.valueChanged[int].connect(self.num_4.display)
        self.sld_5.valueChanged[int].connect(self.num_5.display)
        self.sld_6.valueChanged[int].connect(self.num_6.display)
        
        self.old_value_sld1 = self.sld_def
        self.old_value_sld2 = self.sld_def
        self.old_value_sld3 = self.sld_def
        self.old_value_sld4 = self.sld_def
        self.old_value_sld5 = self.sld_def
        self.old_value_sld6 = self.sld_def
        
    
    def create_checkboxes(self):
        self.checkbox_1 = QCheckBox(self.checkboxes_lables[0], self)
        self.checkbox_2 = QCheckBox(self.checkboxes_lables[1], self)
        
        self.checkbox_1.setChecked(False)                               # Grafik arayüzünde checkboxları oluşturduğumzu fonksiyonuzmu
        self.checkbox_2.setChecked(False)
        
        self.checkbox_1.stateChanged.connect(self.checkboxClicked)
        self.checkbox_2.stateChanged.connect(self.checkboxClicked)
        
        
    def create_buttons(self):
        self.btn_1 = QPushButton(self.btns_lables[0], self)
        self.btn_2 = QPushButton(self.btns_lables[1], self)
        self.btn_3 = QPushButton(self.btns_lables[2], self)             # Grafik arayüzünde butonları oluşturduğumzu fonksiyonuzmu
        
        self.btn_1.clicked.connect(self.buttonClicked)
        self.btn_2.clicked.connect(self.buttonClicked)
        self.btn_3.clicked.connect(self.buttonClicked)
        
        
    def create_graphics(self):              # ses sinyallerini grafiksel olarak görmek için oluşturduğumuz fonksiyon
        figure_1 = plt.figure()
        self.figure_2 = plt.figure()            # 3 figüre içinde yazdırma fonksiyonlarını figürelere atadk
        self.figure_4 = plt.figure()
        
        self.canvas_1 = FigureCanvas(figure_1)
        self.canvas_2 = FigureCanvas(self.figure_2)     # Pyqt5 kütüphanesinde matplotlib ile görselleştirme yapabilmek için canvas oluşturduk
        self.canvas_4 = FigureCanvas(self.figure_4)
            
        self.toolbar_1 = NavigationToolbar(self.canvas_1, self)
        self.toolbar_2 = NavigationToolbar(self.canvas_2, self)
        self.toolbar_4 = NavigationToolbar(self.canvas_4, self)     # canvaslar için toolbar oluşturduk

        figure_1.clear()
        self.figure_2.clear()       # öncedek plot edilmiş görsel olabilmesi sebebiyle canvsaı temizledik
        self.figure_4.clear()
        
        ax_1 = figure_1.add_subplot(1, 1, 1)
        self.ax_2 = self.figure_2.add_subplot(1, 1, 1)  # ekranda subpot yerleşimlerini belirledik
        self.ax_4 = self.figure_4.add_subplot(1, 1, 1)
        
        ax_1.set_xlabel('Sıklık, Hz')
        self.ax_2.set_xlabel('Frekans,Hz')      
        self.ax_4.set_xlabel('Zaman, t')
        
        figure_1.align_xlabels()
        self.figure_2.align_xlabels()           # graphlar için  x ve y eksenindeki labelları belirledik
        self.figure_4.align_xlabels()
        
        ax_1.set_ylabel('Genlik')
        self.ax_2.set_ylabel('Genlik')
        self.ax_4.set_ylabel('Genlik')
        
        figure_1.align_ylabels()
        self.figure_2.align_ylabels()
        self.figure_4.align_ylabels()
    
        self.elem_per_herz = self.spectrum.shape[1] // (self.max_freq - self.min_freq)
        
        ax_1.plot(np.fft.rfftfreq(self.nframes, 1./ self.framerate)[::self.koeff], 
                  np.abs(self.spectrum[0][::self.koeff]) / self.nframes)
                                                                                            # forrier transform
        self.ax_2.plot(np.fft.rfftfreq(self.nframes, 1./ self.framerate)[::self.koeff],
                       np.abs(self.spectrum[0][::self.koeff]) / self.nframes)
        self.ax_4.plot(self.channels[0][::self.koeff])
        
        self.canvas_1.draw()
        self.canvas_2.draw()        # canvas üzerinde çizim işlemini gerçekleştirdik
        self.canvas_4.draw()

        
    def create_interface(self):
        self.labels_box = QHBoxLayout()
        self.labels_box.addWidget(self.label_1)
        self.labels_box.addWidget(self.label_2)
        self.labels_box.addWidget(self.label_3)
        self.labels_box.addWidget(self.label_4)     # grafiklere etiketleri yerleştirdik
        self.labels_box.addWidget(self.label_5)
        self.labels_box.addWidget(self.label_6)
        
        self.nums_box = QHBoxLayout()
        self.nums_box.addWidget(self.num_1)
        self.nums_box.addWidget(self.num_2)
        self.nums_box.addWidget(self.num_3)     # numaraları yerleştirdk
        self.nums_box.addWidget(self.num_4)
        self.nums_box.addWidget(self.num_5)
        self.nums_box.addWidget(self.num_6)
        
        self.slds_box = QHBoxLayout()
        self.slds_box.addWidget(self.sld_1)
        self.slds_box.addWidget(self.sld_2)
        self.slds_box.addWidget(self.sld_3)                     # bu kısım grafik arayüzünde canvas plot çizimleri için gerekli ayarlamaların yapıldığı kısımdır
        self.slds_box.addWidget(self.sld_4)
        self.slds_box.addWidget(self.sld_5)
        self.slds_box.addWidget(self.sld_6)

        self.graphs_box_1 = QVBoxLayout()
        self.graphs_box_1.addWidget(self.toolbar_4)
        self.graphs_box_1.addWidget(self.canvas_4)
        
        self.checks_and_btns_box = QHBoxLayout()
        self.checks_and_btns_box.addWidget(self.checkbox_1)
        self.checks_and_btns_box.addWidget(self.checkbox_2)
        self.checks_and_btns_box.addWidget(self.btn_1)
        self.checks_and_btns_box.addWidget(self.btn_2)
        self.checks_and_btns_box.addWidget(self.btn_3)
        
        self.graphs_box_2 = QVBoxLayout()
        self.graphs_box_2.addWidget(self.toolbar_1)
        self.graphs_box_2.addWidget(self.canvas_1)
        self.graphs_box_2.addWidget(self.toolbar_2)
        self.graphs_box_2.addWidget(self.canvas_2)
        
        self.left_box = QVBoxLayout()
        self.left_box.addLayout(self.labels_box)
        self.left_box.addLayout(self.slds_box)
        self.left_box.addLayout(self.nums_box)
        self.left_box.addLayout(self.graphs_box_1)
        
        self.right_box = QVBoxLayout()
        self.right_box.addLayout(self.checks_and_btns_box)
        self.right_box.addLayout(self.graphs_box_2)
        
        self.all_box = QHBoxLayout()
        self.all_box.addLayout(self.left_box)
        self.all_box.addLayout(self.right_box)
        
        self.setLayout(self.all_box)
        
        self.setWindowTitle(self.app_name)
        self.showMaximized()
        
        
    def sliderChangeValue(self, value):     # bu fonksiyon yukarıda da açıkladığımız gibi her thread işçisine slider değer değişikliği tespiti üzerine iş                                                         atamasının yapıldığı merkezi sistem
        if (self.sender() == self.sld_1):
            self.sld1_worker = Worker(self.music_edit, 0, value)
            self.threadpool.start(self.sld1_worker)
            
        elif (self.sender() == self.sld_2):
            self.sld2_worker = Worker(self.music_edit, 1, value)
            self.threadpool.start(self.sld2_worker)
            
        elif (self.sender() == self.sld_3):
            self.sld3_worker = Worker(self.music_edit, 2, value)
            self.threadpool.start(self.sld3_worker)
            
        elif (self.sender() == self.sld_4):
            self.sld4_worker = Worker(self.music_edit, 3, value)
            self.threadpool.start(self.sld4_worker)
            
        elif (self.sender() == self.sld_5):
            self.sld5_worker = Worker(self.music_edit, 4, value)
            self.threadpool.start(self.sld5_worker)
            
        else:
            self.sld6_worker = Worker(self.music_edit, 5, value)
            self.threadpool.start(self.sld6_worker)
        
            
    def checkboxClicked(self, state):
        if (self.sender() == self.checkbox_1):
            if (state == Qt.Checked):
                self.checkbox_2.setChecked(False)
                self.channels = self.channels_kliping.copy()        # 2 cehckbox'umuz var biri kırpma diğeri Envelope işlemi yapıyor bunların görevlerinin                                                                                  girildiği işlem aşaması
                self.spectrum = self.spectrum_kliping.copy()
            else:
                self.channels = self.channels_original.copy()
                self.spectrum = self.spectrum_original.copy()
                
        else:
            if (state == Qt.Checked):
                self.checkbox_1.setChecked(False)
                self.channels = self.channels_envelop.copy()
                self.spectrum = self.spectrum_envelop.copy()
            else:
                self.channels = self.channels_original.copy()
                self.spectrum = self.spectrum_original.copy()
                
        self.sld_1.setValue(self.sld_def)
        self.sld_2.setValue(self.sld_def)
        self.sld_3.setValue(self.sld_def)
        self.sld_4.setValue(self.sld_def)
        self.sld_5.setValue(self.sld_def)
        self.sld_6.setValue(self.sld_def)
        
        draw_1 = Worker(self.draw_array, self.spectrum, 0)
        self.threadpool.start(draw_1)

        draw_2 = Worker(self.draw_array, self.channels, 1)
        self.threadpool.start(draw_2)
            
        
    def buttonClicked(self):
        if (self.sender() == self.btn_1):
            if (self.music_is_playing == False):
                self.music_is_playing = True
                self.music_worker = Worker(self.start_music)
                self.threadpool.start(self.music_worker)
            
        elif (self.sender() == self.btn_2):
            if (self.music_is_playing == True):             # Koddaki tüm buton işlevlerinin bulunduğu kısım çal,durdur duraklat gibi
                self.music_is_playing = False
            
        else:
            if (self.music_is_playing == True):
                self.music_is_playing = False
                self.threadpool.clear()
                
            sliders = [self.sld1_worker, self.sld2_worker,
                       self.sld3_worker, self.sld4_worker,
                       self.sld5_worker, self.sld6_worker]
            for slider in sliders:
                self.sld_stop(slider)
            
            self.buffer_cnt = 0
            
            self.sld_1.setValue(self.sld_def)
            self.sld_2.setValue(self.sld_def)
            self.sld_3.setValue(self.sld_def)
            self.sld_4.setValue(self.sld_def)
            self.sld_5.setValue(self.sld_def)
            self.sld_6.setValue(self.sld_def)
            
            self.checkbox_1.setChecked(False)
            self.checkbox_2.setChecked(False)
            
            tmp_worker = Worker(self.tmp_func)
            self.threadpool.start(tmp_worker)
            
    
    def sld_stop(self, slider):
        ids = { self.sld1_worker: 0, self.sld2_worker: 1,
                self.sld3_worker: 2, self.sld4_worker: 3,
                self.sld5_worker: 4, self.sld6_worker: 5}               # sliderı bıraktığımızda slidera göre sinyal değiştiren işçileri durdurduğumuz kısım
        
        slider = Worker(self.music_edit, ids[slider], self.sld_def)
        self.threadpool.start(slider)
        
        
    def tmp_func(self):
        while (self.threadpool.activeThreadCount() != 1):
            sleep(0.1)
        self.channels = self.channels_original.copy()
        self.spectrum = self.spectrum_original.copy()               # işçilerin durmasıyla müziğin kesilme durumu
        print('music stopped')


    def start_music(self):
            tmp_channels = []
            tmp_channels.append(self.channels[0][self.buffer_cnt * self.buffer_size:
                                             (self.buffer_cnt + 1) * self.buffer_size + 1:])
            tmp_channels.append(self.channels[1][self.buffer_cnt * self.buffer_size:
                                             (self.buffer_cnt + 1) * self.buffer_size + 1:])   # Müziğin ilk oynatılma veya durdurulduktan sonra yeniden                                                                                                            oynatıldığı kısım
            tmp_channels = np.array(tmp_channels)
            tmp_channels = np.ascontiguousarray(tmp_channels.T)
            tmp_sound = pygame.sndarray.make_sound(tmp_channels)
            
            sound = tmp_sound
            if (self.music_is_playing == False):
                return
            pygame.mixer.Sound.play(sound)
        
            start_pos = self.buffer_cnt
            for self.buffer_cnt in range(start_pos + 1, self.nframes // self.buffer_size):
                tmp_channels = []
                tmp_channels.append(self.channels[0][self.buffer_cnt * self.buffer_size:
                                                     (self.buffer_cnt + 1) * self.buffer_size + 1:])
                tmp_channels.append(self.channels[1][self.buffer_cnt * self.buffer_size:
                                                     (self.buffer_cnt + 1) * self.buffer_size + 1:])
                tmp_channels = np.array(tmp_channels)
                tmp_channels = np.ascontiguousarray(tmp_channels.T)
                tmp_sound = pygame.sndarray.make_sound(tmp_channels)
            
                while (pygame.mixer.get_busy()):
                    sleep(0.01)
                    
                sound = tmp_sound
                if (self.music_is_playing == False):
                    return
                pygame.mixer.Sound.play(sound)
        
            tmp_channels = []
            tmp_channels.append(self.channels[0][self.buffer_cnt * self.buffer_size::])
            tmp_channels.append(self.channels[1][self.buffer_cnt * self.buffer_size::])
            tmp_channels = np.array(tmp_channels)
            tmp_channels = np.ascontiguousarray(tmp_channels.T)
            tmp_sound = pygame.sndarray.make_sound(tmp_channels)
        
            while (pygame.mixer.get_busy()):
                sleep(0.01)
             
            sound = tmp_sound
            if (self.music_is_playing == False):
                return
            pygame.mixer.Sound.play(sound)
            
            self.buffer_cnt = 0
            self.music_is_playing = False
            
            
    def music_edit(self, pos, value):
        old_values = {
            0: self.old_value_sld1,
            1: self.old_value_sld2,
            2: self.old_value_sld3,
            3: self.old_value_sld4,
            4: self.old_value_sld5,         #sliderların eski verisini saklama için açılan tempdeğerleri
            5: self.old_value_sld6
        }
        old_value = old_values[pos]
        
        if pos == 0:
            self.old_value_sld1 = value
        elif pos == 1:
            self.old_value_sld2 = value
        elif pos == 2:
            self.old_value_sld3 = value
        elif pos == 3:
            self.old_value_sld4 = value
        elif pos == 4:
            self.old_value_sld5 = value
        else:
            self.old_value_sld6 = value
        
        if (old_value == value):
            return
        
        if (pos == 0):
            for i in range(self.nchannels):
                self.spectrum[i][:self.elem_per_herz * self.bands[1][pos] + 1] *= 10**((value - old_value) / 20)
                
        elif (pos == 5):
            for i in range(self.nchannels):
                self.spectrum[i][self.elem_per_herz * self.bands[0][pos]:] *= 10**((value - old_value) / 20)
                
        else:
            for i in range(self.nchannels):
                self.spectrum[i][self.elem_per_herz * self.bands[0][pos]:self.elem_per_herz * self.bands[1][pos] +
                                 1] *= 10**((value - old_value) / 20)
        
        self.channels = (np.fft.irfft(self.spectrum)).astype(self.types[self.sampwidth])

        draw_1 = Worker(self.draw_array, self.spectrum, 0)
        self.threadpool.start(draw_1)

        draw_2 = Worker(self.draw_array, self.channels, 1)
        self.threadpool.start(draw_2)
        
        
    def draw_array(self, arr, spectrum_or_channel):     # grafikleri plot etme ksıımını görmüştük bu kısım oranın daha alt bölümü
        if (spectrum_or_channel == 0):
            self.figure_2.clear()
            self.ax_2 = self.figure_2.add_subplot(1, 1, 1)
            self.ax_2.set_xlabel('frekans,Hz')
            self.figure_2.align_xlabels()
            self.ax_2.set_ylabel('Genlik')          
            self.figure_2.align_ylabels()
            self.ax_2.plot(np.fft.rfftfreq(self.nframes, 1./ self.framerate)[::self.koeff],
                       np.abs(arr[0][::self.koeff]) / self.nframes)
            self.canvas_2.draw()
            
        else:
            self.figure_4.clear()
            self.ax_4 = self.figure_4.add_subplot(1, 1, 1)
            self.ax_4.set_xlabel('zaman, t')
            self.figure_4.align_xlabels()
            self.ax_4.set_ylabel('Genlik')
            self.figure_4.align_ylabels()
            self.ax_4.plot(arr[0][::self.koeff])
            self.canvas_4.draw()
    
    
    def doing_kliping(self, channels):
        print('kliping start')
        start_time = time()
        threshold_max = int(0.6 * np.max(channels[0]))
        threshold_min = int(0.6 * np.min(channels[0]))          #grafiği alttan ve üstten kırpma işlemi checkbox ile aktifleşir
        
        self.channels_kliping = np.maximum(np.minimum(channels, threshold_max),
                                           threshold_min).astype(self.types[self.sampwidth])
        self.spectrum_kliping = np.fft.rfft(self.channels_kliping)
        print('kliping end: ' + str(time() - start_time))
    
    
    def doing_envelop(self, channels):
        print('envelop start')
        start_time = time()
        frequency = 1 / 15
        envelope_sig = np.array([abs(sin(2 * pi * frequency * t / self.framerate))  # Müzikte sıklıkça kullanılan envelope işlemi için hazırladğımız bir kod parçası
                                 for t in range(self.nframes)])
        tmp_channels = channels.copy()
        
        for i in range(self.nchannels):
                tmp_channels[i] = (tmp_channels[i] * envelope_sig).astype(self.types[self.sampwidth])
        
        self.channels_envelop = tmp_channels.copy()
        self.spectrum_envelop = np.fft.rfft(self.channels_envelop)
        print('envelop end: ' + str(time() - start_time))


    
if __name__ == '__main__':
    Equalizer = QApplication(sys.argv)      #arayüz için uygulamaları hazırlar
    Main_Window = Main_Window_class()        #ana pencereyi kurar döngü başlar
    sys.exit(Equalizer.exec_())         #kapata basınca işlemleri durdurmak içindir.

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
8
kliping startenvelop start

kliping end: 0.1798229217529297
envelop end: 0.8466987609863281


SystemExit: 0